In [1]:
from qtl_control.station import parse_config_to_station

station = parse_config_to_station("tests/test_station.yaml")

In [2]:
_, test_conf = station.new_configuration("test")

root, {}, {'combined_controller_0': <qtl_control.controller_module.MockCombinedController object at 0x0000014981BB9F70>}
combined_controller_0, {}, {'hw_controller_0': <qtl_control.controller_module.MockHWController object at 0x0000014981AA5D30>, 'abstract_controller_0': <qtl_control.controller_module.MockController object at 0x0000014981BBB650>}
hw_controller_0, {'amplitude': <qtl_control.controller_module.Setting object at 0x0000014981C00590>}, {}
abstract_controller_0, {}, {}


In [3]:
test_conf

{'root': {'combined_controller_0': {'hw_controller_0': {'amplitude': 0},
   'abstract_controller_0': {}}}}

In [4]:
station.change_settings({"combined_controller_0.hw_controller_0.amplitude": 1})

ValueError: not enough values to unpack (expected 2, got 1)

In [5]:
test_conf

{'root': {'combined_controller_0': {'hw_controller_0': {'amplitude': 0},
   'abstract_controller_0': {}}}}